In [37]:
# ----------------------------------------- #
#                  MODULES                  #

# Standard Modules
import os
import warnings
from joblib import Parallel, delayed

# Third-Party Modules
import geopandas as gpd
import h3
import numpy as np
import pandas as pd
import rasterio
import rasterio.features
import rioxarray
from shapely.geometry import box, Point, Polygon
import xarray as xr
import rioxarray
import xarray as xr
import geopandas as gpd
import rasterio.features
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from joblib import Parallel, delayed
import geopandas as gpd
import numpy as np
import h3

warnings.filterwarnings("ignore")

# System Configuration
parallel = Parallel(n_jobs=8)

#                                           #
# ----------------------------------------- #

# ----------------------------------------- #
#                 FUNCTIONS                 #


# Open Data
def open_geotiff(
    path,
    data_crs="EPSG:4326",
    desired_crs="EPSG:4326",
    mask_upper_data=False,
    mask_lower_data=False,
    mask_upper_val=None,
    mask_lower_val=None,
):
    if not os.path.exists(path):
        print(f"WARNING: Path does not exist - {path}")
        return None
    else:
        # Open Data
        da = rioxarray.open_rasterio(path)

        # Filter Data to Sea Level
        if mask_upper_data:
            da = xr.where(da > mask_upper_val, mask_upper_val, da)
        if mask_lower_data:
            da = xr.where(da < mask_upper_val, mask_lower_val, da)

        # Set CRS (example: WGS84 EPSG:4326)
        if da.rio.crs == None:
            da = da.rio.write_crs(data_crs)

        else:
            # reproject CRS
            print("need to add a crs reprojection", da.rio.crs, "->", desired_crs)
            # TODO: reproject to provided crs

        return da


# Open Water Region Polygon
def open_water_polygon_aoi(path, data_crs="EPSG:4326"):
    if not os.path.exists(path):
        print(f"WARNING: Path does not exist - {path}")
        return None
    else:
        # Open Polygon
        polygon_area = gpd.read_parquet(path)

        # Dissolve Geometry
        polygon_area = polygon_area.dissolve()

        # Enforce Projection
        polygon_area = polygon_area.to_crs(data_crs)

        return polygon_area


# Clip Raster Data to AOI
def clip_raster_to_aoi(da, poly_data):
    da_clipped = da.rio.clip(
        poly_data.geometry,  # geometry column
        poly_data.crs,  # CRS of the geometry
        all_touched=True,  # include partial pixels
        drop=True,  # drop pixels outside
    )
    return da_clipped


# Clip Polygon to Bounds
def clip_poly_to_data_bounds(da, poly_data):
    # Your bounds (min_lon, min_lat, max_lon, max_lat)
    da_bounds = da.rio.bounds()

    # Create shapely box (polygon)
    polygon = box(*da_bounds)

    # Make GeoDataFrame
    bounds_gdf = gpd.GeoDataFrame({"geometry": [polygon]}, crs="EPSG:4326")

    # Clip Polygon Data
    poly_data = poly_data.clip(bounds_gdf)

    # Subset to Geometry
    poly_data = poly_data[["geometry"]]

    return poly_data


# Geometry Point to H3 Grid
def point_to_h3(point, res):
    # point: shapely Point geometry
    return h3.latlng_to_cell(point.y, point.x, res)


# H3 Grid to Polygon
def h3_to_polygon(h3_index):
    boundary = h3.cell_to_boundary(h3_index)
    boundary_lonlat = [(lon, lat) for lat, lon in boundary]
    return Polygon(boundary_lonlat)


# Identify All H3 Grids in Polygon
def get_all_h3_cells(poly_area, target_resolution=5):
    # Original Polygon Area
    poly_area_orig = poly_area.copy()

    # Explode Geometries
    poly_area = poly_area.explode()
    poly_area["geometry"] = poly_area.buffer(0.2)

    h3_grids = []
    for geometry_val in poly_area["geometry"]:
        # For each polygon geometry (Multi/Poly)
        hex_ids = h3.geo_to_cells(geometry_val, res=target_resolution)

        h3_grids.extend(hex_ids)

    # Get Unique Grids
    h3_grids = np.unique(h3_grids)

    # Build H3 Grid
    h3_gdf = gpd.GeoDataFrame(
        {
            "h3_index": h3_grids,
            "geometry": [h3_to_polygon(h) for h in h3_grids],
        },
        crs="EPSG:4326",
    )

    # Ensure they Overlap with Polygon Area
    h3_gdf_clipped = h3_gdf.clip(poly_area_orig)
    h3_gdf = h3_gdf[h3_gdf.h3_index.isin(h3_gdf_clipped.h3_index)]
    print("Total Cells:", len(h3_gdf))

    return h3_gdf, h3_gdf_clipped


# Vectorized Polygon Extract
def polygon_stats_vectorized(
    da: xr.DataArray, gdf: gpd.GeoDataFrame
) -> gpd.GeoDataFrame:
    """
    Compute descriptive statistics (min, max, mean, median, std) for raster data within polygons.

    Parameters:
    - da: xarray.DataArray with raster data (must have rio.crs and rio.transform)
    - gdf: geopandas.GeoDataFrame with polygon geometries

    Returns:
    - gpd.GeoDataFrame: Input GeoDataFrame with added columns for min, max, mean, median, and std
    """

    # Validate inputs
    if not hasattr(da, "rio") or da.rio.crs is None:
        raise ValueError("Input DataArray must have a valid CRS")
    if gdf.crs is None:
        raise ValueError("Input GeoDataFrame must have a valid CRS")
    if not all(gdf.geometry.is_valid):
        raise ValueError("GeoDataFrame contains invalid geometries")

    # Ensure CRS match
    if da.rio.crs != gdf.crs:
        gdf = gdf.to_crs(da.rio.crs)

    # Rasterize polygons with unique IDs
    shapes = ((geom, idx) for idx, geom in enumerate(gdf.geometry))
    mask = rasterio.features.rasterize(
        shapes,
        out_shape=(da.rio.height, da.rio.width),
        transform=da.rio.transform(),
        fill=-1,  # Mark areas outside polygons
        all_touched=True,
        dtype=np.int32,
    )

    # Flatten raster and mask
    data_flat = da.values.ravel()
    mask_flat = mask.ravel()

    # Filter valid pixels (exclude nodata and non-polygon areas)
    valid = (mask_flat >= 0) & np.isfinite(data_flat)
    if not np.any(valid):
        raise ValueError("No valid pixels found in the raster for any polygon")

    data_flat = data_flat[valid]
    mask_flat = mask_flat[valid]

    # Sort by polygon ID for efficient grouping
    order = np.argsort(mask_flat)
    data_flat = data_flat[order]
    mask_flat = mask_flat[order]

    # Find group boundaries
    unique_ids, idx_start = np.unique(mask_flat, return_index=True)
    idx_end = np.r_[idx_start[1:], len(mask_flat)]

    # Initialize stats arrays
    n_polygons = len(gdf)
    mins = np.full(n_polygons, np.nan)
    maxs = np.full(n_polygons, np.nan)
    means = np.full(n_polygons, np.nan)
    medians = np.full(n_polygons, np.nan)
    stds = np.full(n_polygons, np.nan)

    # Compute stats for polygons with valid pixels
    valid_indices = np.arange(len(unique_ids))
    mins[unique_ids] = np.minimum.reduceat(data_flat, idx_start)
    maxs[unique_ids] = np.maximum.reduceat(data_flat, idx_start)
    means[unique_ids] = np.add.reduceat(data_flat, idx_start) / (idx_end - idx_start)
    medians[unique_ids] = [
        np.median(data_flat[s:e]) for s, e in zip(idx_start, idx_end)
    ]
    stds[unique_ids] = [
        np.std(data_flat[s:e], ddof=0) for s, e in zip(idx_start, idx_end)
    ]

    # Assemble DataFrame
    stats_df = pd.DataFrame(
        {"min": mins, "max": maxs, "mean": means, "median": medians, "std": stds}
    )

    # Join stats to GeoDataFrame
    return gdf.reset_index(drop=True).join(stats_df)


#                                           #
# ----------------------------------------- #

In [38]:
# Water Polygon Path
water_poly_path = "../../../data/processed/GIS/ocean/TERRITORIAL_WATERS.parquet"

# Data Sourced from GEBCO_2025 - https://download.gebco.net
raster_path = "../../../data/raw/bathymetry/GEBCO_12_Aug_2025_e9cdc1fe517e/gebco_2025_n54.524_s33.96_w-130.0_e-120.0.tif"

In [39]:
# Open Water Region Polygon
water_poly = open_water_polygon_aoi(water_poly_path, data_crs="EPSG:4326")

# Open GeoTiff
da = open_geotiff(
    raster_path, data_crs="EPSG:4326", mask_upper_data=True, mask_upper_val=0
)

# Clip AOI to Polygon AOI
da_clipped = clip_raster_to_aoi(da=da, poly_data=water_poly)

# Clip Polygon to Bounds of Data Array
water_poly = clip_poly_to_data_bounds(da=da_clipped, poly_data=water_poly)

In [40]:
# Desired Resolution
desired_resolution = 6

In [41]:
def h3_from_single_polygon(polygon, res=5):
    return list(h3.geo_to_cells(polygon, res=res))


def get_all_h3_cells(poly_area, target_resolution=5, n_jobs=-1):
    # Explode multipolygons to single polygons
    poly_area = poly_area.explode(ignore_index=True)
    print("...Exploded Polygon")

    # Optional: tiny buffer if needed
    poly_area["geometry"] = poly_area["geometry"].buffer(0.0)
    poly_area["geometry"] = poly_area["geometry"].buffer(0.1)
    print("...Buffered Geometry")

    # Parallel processing for speed
    results = Parallel(n_jobs=n_jobs)(
        delayed(h3_from_single_polygon)(geom, target_resolution)
        for geom in poly_area["geometry"]
    )
    print("...Collected H3 Grids")

    # Flatten list and deduplicate
    h3_ids = np.unique([h for sublist in results for h in sublist])

    # Convert H3 cells to polygons
    h3_polys = [Polygon(h3_to_polygon(h)) for h in h3_ids]
    print("...Built Polygons")

    # Build GeoDataFrame
    h3_gdf = gpd.GeoDataFrame(
        {"h3_index": h3_ids, "geometry": h3_polys}, crs="EPSG:4326"
    )

    # Clip to original area to remove fringe cells
    h3_gdf_clipped = h3_gdf.clip(poly_area)
    print("...Clipped to Bounding Area")

    h3_gdf = h3_gdf[h3_gdf.h3_index.isin(h3_gdf_clipped.h3_index)]
    print("...Filtered Cells")
    print()
    print("Total Cells:", len(h3_gdf))
    return h3_gdf, h3_gdf_clipped

In [42]:
# Get All H3 Polygons
h3_water_poly, h3_water_poly_clipped = get_all_h3_cells(
    poly_area=water_poly, target_resolution=desired_resolution, n_jobs=-1
)

...Exploded Polygon
...Buffered Geometry
...Collected H3 Grids
...Built Polygons
...Clipped to Bounding Area
...Filtered Cells

Total Cells: 7431


In [43]:
h3_water_poly_clipped.explore().save("test.html")

In [44]:
# Get Polygon Statistics for H3 Grid
poly_stats_vectorized = polygon_stats_vectorized(
    da=da_clipped, gdf=h3_water_poly_clipped
)

In [45]:
# Assign to Full H3 Poly
poly_stats_vectorized = poly_stats_vectorized.drop(columns="geometry")

poly_stats_vectorized = pd.merge(poly_stats_vectorized, h3_water_poly, how="left")
poly_stats_vectorized["mean"] = poly_stats_vectorized["mean"].fillna(-9999)

In [46]:
poly_stats_vectorized = gpd.GeoDataFrame(
    poly_stats_vectorized, geometry="geometry", crs="EPSG:4326"
)

In [47]:
poly_stats_vectorized.dropna(subset=["mean", "geometry"]).explore(
    "mean", cmap="Blues_r", vmin=-200, vmax=0  # ocean is a good cmap too
).save(f"gebco_bathymetry_{desired_resolution}.html")

In [48]:
# Save to File
poly_stats_vectorized.to_parquet(
    f"../../../data/processed/GIS/GEBCO_BATHEYMETRY_{desired_resolution}.parquet"
)

In [32]:
h3_water_poly_clipped_ea = h3_water_poly_clipped.copy()
h3_water_poly_clipped_ea["clipped_area"] = h3_water_poly_clipped_ea.area
h3_water_poly_clipped_ea = h3_water_poly_clipped_ea[["h3_index", "clipped_area"]]

h3_water_poly_unclipped = h3_water_poly.copy()
h3_water_poly_unclipped["unclipped_area"] = h3_water_poly_unclipped.area
h3_water_poly_unclipped = h3_water_poly_unclipped[["h3_index", "unclipped_area"]]

tmp = pd.merge(h3_water_poly_clipped_ea, h3_water_poly_unclipped)

In [33]:
tmp["water_covers"] = tmp["clipped_area"] / tmp["unclipped_area"]

h3_water_poly_covers = pd.merge(h3_water_poly, tmp)

In [34]:
h3_water_poly_covers.dropna(subset=["water_covers", "geometry"]).explore(
    "water_covers",
    cmap="turbo",
).save("area_coverage.html")

In [35]:
h3_water_poly_covers = h3_water_poly_covers[["h3_index", "geometry", "water_covers"]]
h3_water_poly_covers["water_covers"] = round(h3_water_poly_covers["water_covers"], 5)
h3_water_poly_covers["H3_RESOLUTION"] = desired_resolution
h3_water_poly_covers.to_parquet(
    f"../../../data/processed/GIS/H3_{desired_resolution}_Polygons_With_Water_Coverage.parquet"
)

In [36]:
# compute distance to shelf break by deriving a contour on a depth threshold (e.g., 200 m) and computing H3-cell distance to that contour. shelf break proximity = killer covariate.
# compute BPI (bathymetric position index) to detect depressions/ridges (use whitebox or scipy filters).
# add seafloor substrate layers if available (gravel/mud/rock) — very useful for some prey.
# compute bathymetric complexity at multiple scales (3×3 window, 25×25 window) and include both as covariates.

1. NOAA National Centers for Environmental Information (NCEI)
Bathymetry Data Viewer has REST-like endpoints behind it, but the public docs focus on downloads.
Direct API for their netCDF/GeoTIFF products isn’t super public, but you can grab prebuilt rasters via:
ERDDAP servers (many NOAA datasets, bathy included)
Example: https://coastwatch.pfeg.noaa.gov/erddap/griddap/
Lets you query depth by bounding box, depth variable = altitude or z.
NCEI also serves Coastal Relief Models (CRM) as OPeNDAP / WCS.

In [ ]:
def polygon_stats_from_rioxarray(da: xr.DataArray, gdf: gpd.GeoDataFrame):
#     """
#     Compute min, max, mean, median, std for each polygon in gdf from a rioxarray.DataArray.

#     Parameters:
#     -----------
#     da : rioxarray.DataArray
#         Raster data array with georeferencing
#     gdf : geopandas.GeoDataFrame
#         Polygons to compute stats for (must have geometry column)

#     Returns:
#     --------
#     pandas.DataFrame : gdf with columns added for stats
#     """
#     # Ensure CRS match
#     if da.rio.crs != gdf.crs:
#         gdf = gdf.to_crs(da.rio.crs)

#     # Rasterize all polygons at once
#     shapes = ((geom, i) for i, geom in enumerate(gdf.geometry))
#     mask = rasterio.features.rasterize(
#         shapes,
#         out_shape=(da.rio.height, da.rio.width),
#         transform=da.rio.transform(),
#         fill=-1,
#         all_touched=True,
#         dtype=int,
#     )

#     mask_da = xr.DataArray(mask, coords=[da.y, da.x], dims=["y", "x"])

#     # Flatten to vectors for fast masking
#     da_flat = da.values.flatten()
#     mask_flat = mask_da.values.flatten()

#     results = []
#     for i in range(len(gdf)):
#         vals = da_flat[mask_flat == i]
#         vals = vals[~np.isnan(vals)]  # ignore nodata
#         if len(vals) == 0:
#             stats_dict = dict(
#                 min=np.nan, max=np.nan, mean=np.nan, median=np.nan, std=np.nan
#             )
#         else:
#             stats_dict = dict(
#                 min=float(vals.min()),
#                 max=float(vals.max()),
#                 mean=float(vals.mean()),
#                 median=float(np.median(vals)),
#                 std=float(vals.std()),
#             )
#         results.append(stats_dict)

#     stats_df = pd.DataFrame(results)
#     return gdf.reset_index(drop=True).join(stats_df)

# # Get Polygon Stats for H3 Grids
# poly_stats = polygon_stats_from_rioxarray(da=da_clipped, gdf=h3_water_poly)